In [1]:
!pip install pennylane
import pennylane as qml
from pennylane import numpy as np

# !pip install numpy
# import numpy as np

!pip install matplotlib
import matplotlib.pyplot as plt


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
dev = qml.device("default.qubit", wires=6)
# wires 0,1,2 are group B (input training data)
# wires 3,4,5 are group A (store past data)

# initialize a, J values (stays same throughout trainig)
a = []
J = np.zeros((6, 6))
for i in range(6):
  a.append( np.random.uniform(-1.0, 1.0) )
  for j in range(i):
    J[i][j] = np.random.uniform(-1.0, 1.0)

# calculate Hint
coeffs = []
obs = []
for j in range(6):
    coeffs.append(a[j])
    obs.append(qml.PauliX(j))
    for k in range(j):
      coeffs.append(J[j][k])
      obs.append(qml.PauliZ(j) @ qml.PauliZ(k))
Hint = qml.Hamiltonian(coeffs, obs) 
# some fixed values (as specified in paper)
D = 3
tau = 0.2

# Parameters
# alpha_array = np.zeros((D, 6))
# beta_array  = np.zeros((D, 6))
# gamma_array = np.zeros((D, 6))
alpha_array = np.random.rand(D, 6)
beta_array  = np.random.rand(D, 6)
gamma_array = np.random.rand(D, 6)
c = 1.0

def encoding(x):
    qml.RY(np.arccos(x), wires=0)
    qml.RY(np.arccos(x), wires=1)
    qml.RY(np.arccos(x), wires=2)

def U1(alpha, beta, gamma, wire):
     qml.RX(alpha, wires=wire)
     qml.RZ(beta, wires=wire)
     qml.RX(gamma, wires=wire)

def evolution(alpha_array, beta_array, gamma_array):
    for i in range (D): # D = 3
        # U1 rotations
        for j in range(6):
            U1(alpha_array[i][j], beta_array[i][j], gamma_array[i][j], j)

        # np.exp(-1.0*j * Hint * tau) # e^(-i Hint tau)
        #change third paramter (larger is more accurate)
        qml.ApproxTimeEvolution(Hint, tau, 5)

@qml.qnode(dev)
def pqc(x, alpha_array, beta_array, gamma_array):
    encoding(x)
    evolution(alpha_array, beta_array, gamma_array)
    
    return [qml.expval(qml.PauliZ(i)) for i in range(3)]

def timestep_output(x, weights):

    # print(qml.draw_mpl(pqc)(0.5))
    result = pqc(x, weights[0], weights[1], weights[2])
    # print("Output of pqc: ", result)
    # print("Type of pqc: ", result[0])
    
    # y = c * np.average(result)
    y = weights[3] * (( result[0] + result[1] + result[2] ) / 3.0)
    # print("Output of each timestep: ", y)

    return y

timestep_output(0.7, [alpha_array, beta_array, gamma_array, c])

# print(qml.draw_mpl(pqc)(0.5, 1))
# print("blahhhhhhhhh", np.arccos(1))


### random code ideas, unused

# expval1 = qml.expval(qml.PauliZ(0))
# expval2 = qml.expval(qml.PauliZ(1))
# expval3 = qml.expval(qml.PauliZ(2))

# avg = (expval1 + expval2 + expval3) / 3.0

# print([expval1.eigvals(), expval2.eigvals(), expval3.eigvals()])

# return c * avg
# return expval1, expval2, expval3

tensor(0.5128401, requires_grad=True)

In [3]:
np.array(0.2)

tensor(0.2, requires_grad=True)

In [4]:
def generate_input(t_in):
    t=8/199*t_in
    if t>=0 and t<=1:
        return -t+0.5
    elif t>=1 and t<=2:
        return t-1.5
    elif t>=2 and t<=3:
        return -t+2.5
    else:
        return t-3.5

train_inputs = []
train_expected_outputs = []

for i in range(100):
    train_inputs.append(generate_input(i))
    train_expected_outputs.append(generate_input(i+1))

print(train_inputs)
print(train_expected_outputs)

[0.5, 0.45979899497487436, 0.41959798994974873, 0.37939698492462315, 0.33919597989949746, 0.2989949748743719, 0.25879396984924624, 0.2185929648241206, 0.17839195979899497, 0.13819095477386933, 0.0979899497487437, 0.05778894472361806, 0.017587939698492483, -0.022613065326633208, -0.06281407035175879, -0.10301507537688448, -0.14321608040201006, -0.18341708542713564, -0.22361809045226133, -0.2638190954773869, -0.3040201005025126, -0.3442211055276382, -0.3844221105527639, -0.42462311557788945, -0.46482412060301503, -0.4949748743718594, -0.4547738693467336, -0.414572864321608, -0.3743718592964824, -0.33417085427135684, -0.29396984924623104, -0.25376884422110546, -0.21356783919597988, -0.1733668341708543, -0.13316582914572872, -0.09296482412060292, -0.05276381909547734, -0.012562814070351758, 0.027638190954773822, 0.06783919597989962, 0.1080402010050252, 0.14824120603015079, 0.18844221105527637, 0.22864321608040195, 0.26884422110552775, 0.3090452261306533, 0.3492462311557789, 0.3894472361809

In [5]:
### Lecture 8 Demo helper functions

def make_predictions(data, model, weights):
    """Predict the labels of all points in a data set for a given model.
    
    Args:
        data (array[float]): Input data. A list with shape N x 2
            representing points on a 2D plane.
        model (qml.QNode): A QNode whose output expectation value will be
            used to make predictions of the labels of data.
        weights (array[float]): The trainable model parameters for the QNode. 
            
    Returns:
        array[int]: The array of predictions for each data point made by 
        the model QNode. 
    """
    preds = []
    
    for idx in range(len(data)):
        estimated_expval = model(data[idx], weights)
        
        preds.append(estimated_expval)
            
    return preds

def make_loss_function(data, labels, model):

    def loss(weights):
        loss_sum = 0.0

        for idx in range(len(data)):
            point = data[idx]
            true_expval = labels[idx]

            estimated_expval = model(point, weights)
            loss_sum += (estimated_expval - true_expval) ** 2

        return loss_sum / len(data)
    
    return loss

def compute_accuracy(predictions, true_labels):
    """Compute the accuracy of our predictions.
    
    Args:
        predictions (array[int]): Predicted values to 
        true_labels (array[int]): Integers identifying the class/label of 
            each data point.
    
    Returns:
        float: Accuracy of the predictions, returned as a percentage.
    """    
    n_samples = len(predictions)
    
    return np.sum(
        [np.abs(predictions[x]-true_labels[x]) <0.2 for x in range(n_samples)
    ]) / n_samples

In [6]:
loss = make_loss_function(train_inputs, train_expected_outputs, timestep_output)

In [12]:
gradient_function = qml.grad(timestep_output)

In [17]:
c=np.asarray(1)
c

tensor(1, requires_grad=True)

In [19]:
#tune this parameter!
opt = qml.GradientDescentOptimizer(stepsize=0.5)

n_its = 100

loss_track = []

# weights = np.array([alpha_array, beta_array, gamma_array, c], requires_grad=True).tolist()
c=np.asarray(1)
weights = [alpha_array, beta_array, gamma_array, c]

for it in range(n_its):
    weights, _loss = opt.step_and_cost(loss, weights)
    if it % 5 == 0:
        our_preds = make_predictions(train_inputs, timestep_output, weights)
        print(weights)
        print(f"Loss at iteration {it} = {_loss}  Accuracy = {compute_accuracy(our_preds, train_expected_outputs)}")
    loss_track.append(_loss)

[tensor([[0.92739594, 0.4634983 , 0.87925563, 0.85938156, 0.00756888,
         0.19693377],
        [0.03505255, 0.97842204, 0.43841039, 0.16225401, 0.61498443,
         0.77639897],
        [0.10562824, 0.07010255, 0.96131515, 0.45860489, 0.87908368,
         0.48235286]], requires_grad=True), tensor([[0.73603176, 0.806028  , 0.93927011, 0.12677213, 0.68323021,
         0.30914075],
        [0.33354588, 0.26897293, 0.85515663, 0.38332603, 0.03342353,
         0.07498794],
        [0.72622248, 0.98973158, 0.8463469 , 0.01552523, 0.157775  ,
         0.52461688]], requires_grad=True), tensor([[0.5072742 , 0.1632863 , 0.19346278, 0.93226726, 0.24432505,
         0.04693771],
        [0.63725238, 0.34748556, 0.56866304, 0.45537562, 0.52110414,
         0.60019291],
        [0.446967  , 0.68781646, 0.35430552, 0.26033006, 0.44227997,
         0.79367707]], requires_grad=True), tensor(1, requires_grad=True)]
Loss at iteration 0 = 0.37434502876629167  Accuracy = 0.1
[tensor([[0.92739594, 0.4

KeyboardInterrupt: ignored

In [ ]:
weights

In [ ]:
# weights = np.random.normal(size=(2, 3), requires_grad=True)
weights = np.array([alpha_array, beta_array, gamma_array, c], requires_grad=True).tolist()
weights